In [1]:
%load_ext autoreload

# Traces simulator v2

In [2]:
%autoreload
from tracer.simulator import TraceSimulator

sim = TraceSimulator(
    number_towers=100,
    number_users=2,
    number_cycles=1000,
)

In [ ]:
sim.generate()

In [ ]:
sim.plot_towers()

In [ ]:
sim.plot_user_trace(user_id=1)